In [1]:
library(tidyverse)

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.2 --
v ggplot2 3.4.0      v purrr   0.3.4 
v tibble  3.1.6      v dplyr   1.0.10
v tidyr   1.2.0      v stringr 1.4.1 
v readr   2.1.2      v forcats 0.5.1 
-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


In [2]:
sales_Data <- read.csv('competitive-data-science-predict-future-sales/sales_train.csv')
testData <- read.csv('competitive-data-science-predict-future-sales/test.csv')
itemsData <- read.csv('competitive-data-science-predict-future-sales/items.csv')
shopsData <- read.csv('competitive-data-science-predict-future-sales/shops.csv')
itemsCatData <- read.csv('competitive-data-science-predict-future-sales/item_categories.csv')

In [3]:
sales_Data

date,date_block_num,shop_id,item_id,item_price,item_cnt_day
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
02.01.2013,0,59,22154,999.00,1
03.01.2013,0,25,2552,899.00,1
05.01.2013,0,25,2552,899.00,-1
06.01.2013,0,25,2554,1709.05,1
15.01.2013,0,25,2555,1099.00,1
10.01.2013,0,25,2564,349.00,1
02.01.2013,0,25,2565,549.00,1
04.01.2013,0,25,2572,239.00,1
11.01.2013,0,25,2572,299.00,1


In [6]:
monthly_sales<-sales_Data%>%group_by(date_block_num,shop_id,item_id)%>%summarise(min_date=min(date),max_date=max(date),mean_price=mean(item_price),item_cnt_month=sum(item_cnt_day))
monthly_sales

`summarise()` has grouped output by 'date_block_num', 'shop_id'. You can override using the `.groups` argument.


date_block_num,shop_id,item_id,min_date,max_date,mean_price,item_cnt_month
<int>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>
0,0,32,03.01.2013,31.01.2013,221.0,6
0,0,33,03.01.2013,28.01.2013,347.0,3
0,0,35,31.01.2013,31.01.2013,247.0,1
0,0,43,31.01.2013,31.01.2013,221.0,1
0,0,51,13.01.2013,31.01.2013,128.5,2
0,0,61,10.01.2013,10.01.2013,195.0,1
0,0,75,17.01.2013,17.01.2013,76.0,1
0,0,88,16.01.2013,16.01.2013,76.0,1
0,0,95,06.01.2013,06.01.2013,193.0,1


In [14]:
ts<-sales_Data%>%group_by(date_block_num)%>%transmute(date_block_num,n=sum(item_cnt_day))%>%distinct(date_block_num,n)
ts

date_block_num,n
<int>,<dbl>
0,131479
1,128090
2,147142
3,107190
4,106970
5,125381
6,116966
7,125291
8,133332
